In [7]:
using Random, CategoricalArrays, DataFrames, DataFramesMeta, StatsBase, CSV
println("##########################################################################################")
println("##########################################################################################")
println("Starting prediction pipeline for penetrating injuries only")
println("##########################################################################################")
println("##########################################################################################")
#data_path = "../Data/time_split_per_injury/penetrating/"
data_path = "/pool001/htazi/Trauma/time_split_per_injury/penetrating/"
train_X_time = CSV.read(data_path*"trauma_X_train_time_penetrating.csv")
test_X_time = CSV.read(data_path*"trauma_X_test_time_penetrating.csv")

train_y_mortality_time = CSV.read(
    data_path*"trauma_y_train_mortality_time_penetrating.csv",
    header=false
)
test_y_mortality_time = CSV.read(
    data_path*"trauma_y_test_mortality_time_penetrating.csv",
    header=false
)
train_y_mortality_time = convert(Matrix, train_y_mortality_time)[:,1]
test_y_mortality_time = convert(Matrix, test_y_mortality_time)[:,1]

train_y_morbidity_time = CSV.read(
    data_path*"trauma_y_train_morbidity_time_penetrating.csv",
    header=false
)
test_y_morbidity_time = CSV.read(
    data_path*"trauma_y_test_morbidity_time_penetrating.csv",
    header=false
)
train_y_morbidity_time = convert(Matrix, train_y_morbidity_time)[:,1]
test_y_morbidity_time = convert(Matrix, test_y_morbidity_time)[:,1]
println("Loaded all data with time split")

severity_categorical_var = [
    :Face_severity,
    :Neck_severity,
    :Thorax_severity,
    :Abdomen_severity,
    :Spine_severity,
    :Upper_Extremity_severity,
    :Lower_Extremity_severity,
    :Pelvis_Perineum_severity
]
for col_name in severity_categorical_var
    train_X_time[!, col_name] = CategoricalArray(train_X_time[!, col_name], ordered=true)
    test_X_time[!, col_name] = CategoricalArray(test_X_time[!, col_name], ordered=true)
    println("Transformed column '", col_name, "' to Ordered Categorical")
end
categorical!(train_X_time)
categorical!(test_X_time)
println("Transformed all other non-ordered Categorical Variables")

# Keep all columns except alcohol
names_with_alcohol = [
    :age, :alcohol,:gender, :race1, :acslevel, :tmode1, :signsoflife,
    :sbp1, :pulse1, :oxysat1, :temp1, :gcstot1, :bleeding_disorder,
    :current_chemotherapy, :congestive_heart_failure, :current_smoker,
    :chronic_renal_failure, :history_cva, :diabetes, :disseminated_cancer,
    :copd, :steroid, :cirrhosis, :history_MI, :history_pvd, :hypertension_medication,
    :method_of_injury, :Face_severity, :Neck_severity, :Thorax_severity, :Abdomen_severity,
    :Spine_severity, :Upper_Extremity_severity, :Lower_Extremity_severity,
    :Pelvis_Perineum_severity, :External_severity, :hemorrhage_ctrl_type
]
train_X_time = train_X_time[!, names_with_alcohol]
test_X_time = test_X_time[!, names_with_alcohol]
#println("Dropped alcohol variable")
println("##########################################################################################")
println("##########################################################################################")



# Model results output
df_final_results_time_split = DataFrame(id = Any[], seed = Any[],
          depth = Any[], minbucket = Any[], criterion = Any[], cp = Any[],
          auc_train = Float64[], auc_valid = Float64[], auc_test = Float64[])

function oct(train_X, test_X, train_y, test_y, seed, outcome, minbucket)
    id = "seed=$(seed)___outcome=$(outcome)___minbucket=$(minbucket)___injury=penetrating"
    outcome = "$(outcome)"
    # Process Data
    Random.seed!(seed)
    @show size(train_X)
    @show size(test_X)


    println("Running OCT for: $(id)")
    # Run Optimal Classification Trees

    # Set of OCT learner and training grid
    oct_lnr = IAI.OptimalTreeClassifier(
        ls_num_tree_restarts=60,
        random_seed=seed,
        treat_unknown_level_missing=true,
        minbucket=minbucket,
        missingdatamode=:separate_class
    )
    grid = IAI.GridSearch(
        oct_lnr,
        max_depth = 8:10,
        criterion = :gini,
    )
    println("Started Fitting the OCT grid search for $(outcome)...")
    IAI.fit!(grid, train_X, train_y, validation_criterion=:auc)
    println("Finished Fitting the OCT grid search for $(outcome)")
    lnr = IAI.get_learner(grid)

    println("Chosen Parameters:")
    for (param, val) in grid.best_params
            println("$(param): $(val)")
    end

    train_auc = IAI.score(grid, train_X, train_y, criterion=:auc)
    test_auc = IAI.score(grid, test_X, test_y, criterion=:auc)

    println("OCT-1 Results of tree $(id) ")
    println("----------------------")
    println("Max Depth $(id) = ", grid.best_params[:max_depth])
    println("Training AUC $(id) = ", round(100 * train_auc, digits=3), "%")
    println("Testing AUC $(id)  = ", round(100 * test_auc, digits=3), "%")

    lnr = IAI.get_learner(grid)
    IAI.show_in_browser(IAI.ROCCurve(lnr, test_X, test_y))

    # Save the tree
    IAI.write_json(joinpath(@__DIR__, "outputs/$(id)_auc$(round(Int, test_auc * 1000)).json"),
        lnr)

    return [
        id, seed, grid.best_params[:max_depth], minbucket,
        grid.best_params[:criterion], grid.best_params[:cp],
        train_auc, train_auc, test_auc
    ]
end


minbucket = 100
seed = 1
# Fit OCTs for mortality
oct_results_mort = oct(
    train_X_time,
    test_X_time,
    train_y_mortality_time,
    test_y_mortality_time,
    seed,
    "hosp_mortality",
    minbucket
)
push!(df_final_results_time_split, vcat(oct_results_mort))

oct_results_morb = oct(
    train_X_time,
    test_X_time,
    train_y_morbidity_time,
    test_y_morbidity_time,
    seed,
    "hosp_morbidity",
    minbucket
)
push!(df_final_results_time_split, vcat(oct_results_morb))
CSV.write("./outputs/results_time_split_penetrating_with_alcohol.csv", df_final_results_time_split)

##########################################################################################
##########################################################################################
Starting prediction pipeline for penetrating injuries only
##########################################################################################
##########################################################################################
Loaded all data with time split
Transformed column 'Face_severity' to Ordered Categorical
Transformed column 'Neck_severity' to Ordered Categorical
Transformed column 'Thorax_severity' to Ordered Categorical
Transformed column 'Abdomen_severity' to Ordered Categorical
Transformed column 'Spine_severity' to Ordered Categorical
Transformed column 'Upper_Extremity_severity' to Ordered Categorical
Transformed column 'Lower_Extremity_severity' to Ordered Categorical
Transformed column 'Pelvis_Perineum_severity' to Ordered Categorical
Transformed all other non-ordered Categori

Training trees...100%|██████████████████████████████████| Time: 0:00:04
Training trees...100%|██████████████████████████████████| Time: 0:00:04
Training trees...100%|██████████████████████████████████| Time: 0:00:05
Training trees...100%|██████████████████████████████████| Time: 0:00:10


Finished Fitting the OCT grid search for hosp_mortality
Chosen Parameters:
cp: 5.820567621483974e-5
criterion: gini
max_depth: 9
OCT-1 Results of tree seed=1___outcome=hosp_mortality___minbucket=100___injury=penetrating 
----------------------
Max Depth seed=1___outcome=hosp_mortality___minbucket=100___injury=penetrating = 9
Training AUC seed=1___outcome=hosp_mortality___minbucket=100___injury=penetrating = 92.34%
Testing AUC seed=1___outcome=hosp_mortality___minbucket=100___injury=penetrating  = 88.432%
size(train_X) = (2000, 37)
size(test_X) = (500, 37)
Running OCT for: seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating
Started Fitting the OCT grid search for hosp_morbidity...


Training trees...100%|██████████████████████████████████| Time: 0:00:05
Training trees...100%|██████████████████████████████████| Time: 0:00:04
Training trees...100%|██████████████████████████████████| Time: 0:00:04
Training trees...100%|██████████████████████████████████| Time: 0:00:06


Finished Fitting the OCT grid search for hosp_morbidity
Chosen Parameters:
cp: 0.001529137528411939
criterion: gini
max_depth: 8
OCT-1 Results of tree seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating 
----------------------
Max Depth seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating = 8
Training AUC seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating = 75.769%
Testing AUC seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating  = 72.081%


"./outputs/results_time_split_penetrating_with_alcohol.csv"

In [20]:
println(df_final_results_time_split)

2×9 DataFrame
│ Row │ id                                                                   │ seed │ depth │ minbucket │ criterion │ cp         │ auc_train │ auc_valid │ auc_test │
│     │ Any                                                                  │ Any  │ Any   │ Any       │ Any       │ Any        │ Float64   │ Float64   │ Float64  │
├─────┼──────────────────────────────────────────────────────────────────────┼──────┼───────┼───────────┼───────────┼────────────┼───────────┼───────────┼──────────┤
│ 1   │ seed=1___outcome=hosp_mortality___minbucket=100___injury=penetrating │ 1    │ 9     │ 100       │ gini      │ 5.82057e-5 │ 0.923404  │ 0.923404  │ 0.884318 │
│ 2   │ seed=1___outcome=hosp_morbidity___minbucket=100___injury=penetrating │ 1    │ 8     │ 100       │ gini      │ 0.00152914 │ 0.757694  │ 0.757694  │ 0.720814 │
